In [1]:
import unittest
from datetime import datetime, timedelta
from db_controller import *

In [20]:
# Controller ve test tabloları oluşturma

controller = DatabaseController()

controller._cursor.executescript("""
        CREATE TABLE Book (
            ISBN TEXT PRIMARY KEY,
            Title TEXT,
            Authors TEXT,
            Description TEXT,
            Genre TEXT,
            Availability INTEGER
        );
        CREATE TABLE Users (
            UserID INTEGER PRIMARY KEY AUTOINCREMENT,
            Name TEXT,
            FavouriteGenre TEXT,
            Password TEXT,
            Fine REAL
        );
        CREATE TABLE Comment (
            CommentID INTEGER PRIMARY KEY AUTOINCREMENT,
            UserID INTEGER,
            BookISBN TEXT,
            CommentText TEXT,
            CommentDate TEXT,
            FOREIGN KEY(UserID) REFERENCES Users(UserID),
            FOREIGN KEY(BookISBN) REFERENCES Book(ISBN)
        );
        CREATE TABLE Reservation (
            ReservationID INTEGER PRIMARY KEY AUTOINCREMENT,
            UserID INTEGER,
            BookISBN TEXT,
            ReservationDate TEXT,
            DueDate TEXT,
            Status TEXT,
            FOREIGN KEY(UserID) REFERENCES Users(UserID),
            FOREIGN KEY(BookISBN) REFERENCES Book(ISBN)
        );
        CREATE TABLE Notification (
            NotificationID INTEGER PRIMARY KEY AUTOINCREMENT,
            UserID INTEGER,
            Message TEXT,
            NotificationDate TEXT,
            FOREIGN KEY(UserID) REFERENCES Users(UserID)
        );
        CREATE TABLE ReadingList (
            ReadingListID INTEGER PRIMARY KEY AUTOINCREMENT,
            UserID INTEGER,
            BookISBN TEXT,
            FOREIGN KEY(UserID) REFERENCES Users(UserID),
            FOREIGN KEY(BookISBN) REFERENCES Book(ISBN)
        );
        """)

controller._conn.commit()

In [21]:
def test_add_book():
        # Test adding a book
        result = controller.add_book(
            isbn='1234567890',
            title='Test Book',
            authors='Author One, Author Two',
            description='This is a test book.',
            genre='Fiction',
            availability=1
        )
        assertTrue(result)
        # Verify the book was added
        controller._cursor.execute("SELECT * FROM Book WHERE ISBN = ?", ('1234567890',))
        book = controller._cursor.fetchone()
        assertIsNotNone(book)

test_add_book()

NameError: name 'assertTrue' is not defined

In [ ]:
def test_search_books(self):
        # Add a book to search
        self.test_add_book()
        # Search by title
        results = self.db_controller.search_books(title='Test')
        self.assertEqual(len(results), 1)
        # Search by author
        results = self.db_controller.search_books(author='Author One')
        self.assertEqual(len(results), 1)
        # Search by genre
        results = self.db_controller.search_books(genre='Fiction')
        self.assertEqual(len(results), 1)
        # Search with no results
        results = self.db_controller.search_books(title='Nonexistent')
        self.assertEqual(len(results), 0)


In [ ]:
def test_add_user(self):
        result = self.db_controller.add_user(
            name='Test User',
            password='password123',
            favourite_genre='Fiction'
        )
        self.assertTrue(result)
        # Verify the user was added
        self.db_controller._cursor.execute("SELECT * FROM Users WHERE Name = ?", ('Test User',))
        user = self.db_controller._cursor.fetchone()
        self.assertIsNotNone(user)

In [ ]:
def test_add_comment(self):
        # Add user and book first
        self.test_add_user()
        self.test_add_book()
        # Get user ID
        self.db_controller._cursor.execute("SELECT UserID FROM Users WHERE Name = ?", ('Test User',))
        user_id = self.db_controller._cursor.fetchone()[0]
        # Add comment
        result = self.db_controller.add_comment(
            user_id=user_id,
            book_isbn='1234567890',
            comment_text='Great book!'
        )
        self.assertTrue(result)
        # Verify the comment was added
        self.db_controller._cursor.execute("SELECT * FROM Comment WHERE UserID = ? AND BookISBN = ?", (user_id, '1234567890'))
        comment = self.db_controller._cursor.fetchone()
        self.assertIsNotNone(comment)

In [ ]:
def test_update_fine(self):
        # Add user first
        self.test_add_user()
        # Get user ID
        self.db_controller._cursor.execute("SELECT UserID FROM Users WHERE Name = ?", ('Test User',))
        user_id = self.db_controller._cursor.fetchone()[0]
        # Update fine
        result = self.db_controller.update_fine(user_id, 10.0)
        self.assertTrue(result)
        # Verify fine is updated
        self.db_controller._cursor.execute("SELECT Fine FROM Users WHERE UserID = ?", (user_id,))
        fine = self.db_controller._cursor.fetchone()[0]
        self.assertEqual(fine, 10.0)

In [ ]:
def test_create_book_report(self):
        # Add user, book, comment, and reservation
        self.test_add_user()
        self.test_add_book()
        self.test_add_comment()
        # Add reservation
        self.db_controller._cursor.execute("SELECT UserID FROM Users WHERE Name = ?", ('Test User',))
        user_id = self.db_controller._cursor.fetchone()[0]
        due_date = datetime.now() + timedelta(days=7)
        result = self.db_controller.add_reservation(
            user_id=user_id,
            book_isbn='1234567890',
            due_date=due_date
        )
        self.assertTrue(result)
        # Create book report
        report = self.db_controller.createBookReport('1234567890')
        self.assertEqual(len(report), 1)
        report_data = report[0]
        self.assertEqual(report_data[0], 'Test Book')  # Title
        self.assertEqual(report_data[4], 1)  # ReservationCount
        self.assertEqual(report_data[5], 1)  # CommentCount

In [ ]:
def test_create_user_report(self):
        # Add user, comment, and reservation
        self.test_add_user()
        self.test_add_book()
        self.test_add_comment()
        self.test_update_fine()
        # Add reservation
        self.db_controller._cursor.execute("SELECT UserID FROM Users WHERE Name = ?", ('Test User',))
        user_id = self.db_controller._cursor.fetchone()[0]
        due_date = datetime.now() + timedelta(days=7)
        self.db_controller.add_reservation(
            user_id=user_id,
            book_isbn='1234567890',
            due_date=due_date
        )
        # Create user report
        report = self.db_controller.createUserReport(user_id)
        self.assertEqual(len(report), 1)
        report_data = report[0]
        self.assertEqual(report_data[0], 'Test User')  # Name
        self.assertEqual(report_data[2], 1)  # ReservationCount
        self.assertEqual(report_data[3], 1)  # CommentCount


In [18]:
controller._conn.close()
del controller